### 模型构造

#### 回顾一下多层感知机的简洁实现中含单隐藏层的多层感知机的实现方法。我们首先构造Sequential实例，然后依次添加两个全连接层。其中第一层的输出大小为256，即隐藏层单元个数是256，第二层输出大小是10，即输出单元个数是10，我们在上一章的其他节中也使用了Sequential类构造模型。这里我们介绍另外一种基于Module类的模型构造方法。他让模型构造更加灵活

### 继承Module类来构造模型

#### Module类是nn模块里提供的一个模型构造类，是所有神经网络模块的基类，我们可以继承它来定义我们想要的模型。下面继承Module类构造本节开头提到的多层感知机。这里定义的MLP类重载了Module类的__init__函数和forward函数。他们分别用于创建模型参数和定义前向计算。正向传播

In [1]:
import torch
from torch import nn
class MLP(nn.Module):
    #声明带有模型参数的层，这里声明了两个全连接层
    def __init__(self,**kwargs):
        #调用MLP父类Module的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
        #参数，如 模型参数的访问，初始化和共享 一节将介绍的模型参数Params
        super(MLP,self).__init__(**kwargs)
        self.hidden = nn.Linear(784,256)# 隐藏层
        self.act = nn.ReLU()
        self.output = nn.Linear(256,10)#输出层
    
    #定义模型的前向计算，即如何根据输入X计算返回所需要的模型
    def forward(self,x):
        a = self.act(self.hidden(x))
        return self.output(a)
        

#### 以上的MLP类中无需定义反向传播函数。系统将通过自动求梯度而自动生成反向传播所需的backward函数

#### 我们可以实例化MLP类得到模型变量net 下面的代码初始化net并传入输入数据X做一次前向计算。其中，net(X)会调用MLP继承自Module类的__call__函数，这个函数将调用MLP类定义的forward函数来完成前向计算

In [2]:
X = torch.rand(2,784)
net = MLP()
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.0191, -0.0409, -0.0760, -0.0655,  0.0334,  0.2557,  0.1012,  0.1000,
          0.0802, -0.0013],
        [ 0.0308, -0.0740, -0.0388,  0.0368, -0.0103,  0.2430,  0.0336,  0.0415,
          0.0891,  0.0786]], grad_fn=<AddmmBackward>)

#### 注意，这里并没有将Module类命名为Layer（层）或者Model（模型）之类的名字，这是因为该类是一个可供自由组建的部件。它的子类既可以是一个层（如PyTorch提供的Linear类），又可以是一个模型（如这里定义的MLP类），或者是模型的一个部分。我们下面通过两个例子来展示它的灵活性。

### Module的子类

#### Module类是一个通用的部件。事实上 ，pytorch还实现了继承自Module的可以方便构建模型的类：如Sequential ModuleList ModuleDict等等

### Sequential类
#### 当模型的前向计算为简单串联各个层的计算时，Sequential类可以通过更加简单的方式定义模型。这正是Sequential类的目的：它可以接收一个子模块的有序字典OrderedDict或者一系列子模块作为参数来逐一添加Module实例，而模型的前向计算就是将这些实例按照添加的顺序逐一计算

#### 下面我们实现一个与Sequential类有相同功能的MySequential类。这或许可以帮助我们更加理解Sequential类的工作机制

In [6]:
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self,*args):
        super(MySequential,self).__init__()
        if len(args) == 1 and isinstance(args[0],OrderedDict):#如果传入的是一个OrderedDict
            for key,module in args[0].item():
                self.add_module(key,module)#add_module方法会将module添加进self._modules(一个OrderedDict)
        else:#传入的是一些Module
            for idx,module in enumerate(args):
                self.add_module(str(idx),module)
    def forward(self,input):
        #self._modules返回一个OrderedDict，保证会按照成员的添加时的顺序遍历成员
        for module in self._modules.values():
            input = module(input)
        return input
            

#### 我们用MySequential类来实现前面描述的MLP类，并使用随机初始化的模型做一次前向计算

In [7]:
net = MySequential(
    nn.Linear(784,256),
    nn.ReLU(),
    nn.Linear(256,10),
    )
print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.1138,  0.0196,  0.0357,  0.0468,  0.0727, -0.0562, -0.0111,  0.0007,
         -0.1291,  0.0688],
        [-0.0229, -0.0164,  0.1022, -0.0028,  0.0512, -0.0318,  0.1002,  0.0834,
         -0.1496,  0.0530]], grad_fn=<AddmmBackward>)

### ModuleList类
#### ModuleList接收一个子模块的列表作为输入，然后也可以类似List那样进行append和extend操作

In [9]:
net = nn.ModuleList([nn.Linear(784,256),nn.ReLU()])
net.append(nn.Linear(256,10))
print(net)

ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


#### 既然Sequential和ModuleList都可以进行列表化构造网络，那么二者的区别是什么呢。ModuleList仅仅是一个储存各种模块的列表，这些模块之间没有联系也没有顺序，（所以不用保证相邻层的输出输入维度匹配）而且没有实现forward功能需自己实现，所以上面执行net(torch.zero(1,784)会报NotImplementedError而Sequential内的模块需要按照顺讯排列，要保证相邻层的输入输出大小匹配，内部forward功能已经实现 ModuleList的出现只是让网络定义前向传播时更加灵活，见下面官网的例子

In [10]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule,self).__init__()
        self.Linears = nn.ModuleList([nn.Linear(10,10) for i in range(10)])
    def forward(self,x):
        #ModuleList can act as an iterable,or be indexed using ints
        for i,l in enumerate(self.Linears):
            x = self.Linears[i // 2](x) + l(x)
        return x

#### 另外，ModuleList不同于一般的Python的list,加入到ModuleList里面的所有模块的参数会被自动添加到整个网络中，

In [13]:
class Module_ModuleList(nn.Module):
    def __init__(self):
        super(Module_ModuleList,self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10,10)])
class Module_List(nn.Module):
    def __init__(self):
        super(Module_List,self).__init__()
        self.linears = [nn.Linear(10,10)]
net1 = Module_ModuleList()
net2 = Module_List()

print("net1:")
for p in net1.parameters():
    print(p.size())

print("net2:")
for p in net2.parameters():
    print(p.size())
    

net1:
torch.Size([10, 10])
torch.Size([10])
net2:


### ModuleDict类
#### ModuleDict接收一个子模块的字典作为输入，然后也可以类似字典那样进行添加访问操作

In [14]:
net = nn.ModuleDict({
    'linear':nn.Linear(784,256),
    'act':nn.ReLU(),
}
)
net['output'] = nn.Linear(256,10) #添加
print(net['linear'])#访问
print(net.output)
print(net)


Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


#### 和ModuleList一样，ModuleDict实例仅仅是存放了一些模块的字典，并没有定义forward函数需要自己定义，同样，ModuleDict也与python的Dict有所不同，ModuleDict 里的所有模块的参数会被自动添加到整个网络中

### 构造复杂的模型

#### 虽然上面介绍的这些类可以使模型构造更加简单，且不需要定义forward函数，但直接继承Module类可以极大的拓展模型构造的灵活性，下面我们构造一个稍微复杂点的网络FancyMLP 在这个网络中，我们通过get_constant函数创建训练中不被迭代的参数，即常数参数，在前向计算中，除了使用创建的常数参数外，我们还使用Tensor的函数和Python的控制流，并多次调用相同的层

In [27]:
class FancyMLP(nn.Module):
    def __init__(self,**kwargs):
        super(FancyMLP,self).__init__(**kwargs)
        
        self.rand_weight = torch.rand((20,20),requires_grad =False)
        self.linear = nn.Linear(20,20)
        
    def forward(self,x):
        x = self.linear(x)
        #使用创建的常数参数，以及nn.funtional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x,self.rand_weight.data)+1)
        
        #复用全连接层，等价于两个全连接层共享参数
        x = self.linear(x)
        #控制流，这里我们需要调用item函数来返回标量进行比较
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()
        
    

#### 在这个FancyMLP模型中，我们使用了常数权重rand_weight，做了矩阵乘法操作，并重复使用了相同的Linear层。下面我们来测试该模型的前向计算

In [31]:
X  = torch.rand(2,20)
net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(4.0679, grad_fn=<SumBackward0>)

#### 因为FancyMLP和Sequential类都是Module类的子类，所以我们可以嵌套调用他们

In [34]:
class NestMLP(nn.Module):
    def __init__(self,**kwargs):
        super(NestMLP,self).__init__(**kwargs)
        self.net = nn.Sequential(nn.Linear(40,30),nn.ReLU())
        
    def forward(self,x):
        return self.net(x)
net = nn.Sequential(NestMLP(),nn.Linear(30,20),FancyMLP())

X = torch.rand(2,40)
print(net)
net(X)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(-1.7600, grad_fn=<SumBackward0>)

### 小结
#### 可以通过继承Module来构造模型
#### Sequential ModuleList ModuleDict类都继承自Module类
#### 与Sequential 不同，ModuleList 和 ModuleDict并没有定义一个完整的网络，他们只是将不同的模块存放在一起，需要自己定义forward函数
#### 虽然Sequential等类可以使模型构造更加简单，但是直接继承Module类可以极大的拓展模型构造的灵活性